##### Copyright 2025 Google LLC.

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

### 1.3: Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

### 1.4: Helper functions

We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [ ]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

In [ ]:
# ================================
# Full Kaggle Notebook: ADK + Gemini 1.5
# Sequential Agents: PatientInput -> FD Auditor -> Centralized Risk
# Outputs: /kaggle/working/centralized_risk_predictions.csv + visualizations
# ================================

# Cell 1: Install / imports (if needed)
# (If ADK is already installed in your Kaggle environment, skip installs)
# !pip install --upgrade google-adk pandas seaborn matplotlib --quiet

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging
from pydantic import ConfigDict

# ADK imports (assumes ADK is available in your environment)
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.apps import App

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
print("✅ Core imports complete.")


In [ ]:
# Cell 2: Initialize Gemini LLM (ADK model wrapper)
# If your environment is configured with proper credentials, this will call the real Gemini.
# We keep temperature moderate; change as you like.
gemini_llm = Gemini(model="gemini-1.5", temperature=0.6)
print("✅ Gemini LLM object created (gemini-1.5).")


In [ ]:
# Cell 3: Helper - safe LLM call wrapper (falls back to mock on failure)
def safe_llm_predict(llm, prompt, fallback_type="audit"):
    """
    Call llm.predict(prompt). If it fails (e.g., auth), return a deterministic mock.
    fallback_type: "audit" or "risk" controls mock format.
    """
    try:
        # ADK Gemini LLM: use .predict(prompt)
        resp = llm.predict(prompt)
        # ADK responses can be string or structured; cast to str for storing
        return resp
    except Exception as e:
        logging.warning(f"LLM call failed; using mock fallback. Error: {e}")
        # deterministic mock behavior:
        if fallback_type == "audit":
            return {"Valid": True, "Comment": "Mock: data looks consistent"}
        else:
            # risk fallback
            return {"FinalRisk": np.random.choice(["High", "Low"], p=[0.3, 0.7])}


In [ ]:
# Cell 4: Define Agents (ADK-compatible)
from google.adk import Agent
from pydantic import ConfigDict
class PatientInputAgent(Agent):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def __init__(self, llm, name="PatientInputAgent"):
        super().__init__(name=name)
        # store llm privately (allowed by model_config)
        self._llm = llm

    # NOTE: runner.run(...) will pass a dict to this method; param name should match keys
    def run(self, num_patients: int = 20):
        patients = []
        for i in range(1, num_patients + 1):
            record = {
                "PatientID": i,
                "BMI": round(np.random.uniform(18, 35), 1),
                "Blood Glucose": int(np.random.randint(70, 200)),
                "BP": f"{np.random.randint(90,160)}/{np.random.randint(60,100)}",
                "Cholesterol": int(np.random.randint(150, 300)),
                "Meals": np.random.choice(["Low Carb", "High Protein", "Balanced", "Vegan"])
            }
            patients.append(record)
        logging.info(f"PatientInputAgent: generated {len(patients)} records.")
        return patients


class FDAuditorAgent(Agent):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def __init__(self, llm, name="FDAuditorAgent"):
        super().__init__(name=name)
        self._llm = llm

    # expects inputs: patient_data (list of dict)
    def run(self, patient_data):
        reviewed = []
        for rec in patient_data:
            prompt = (
                "You are an auditor. Review the following diabetic patient record for obvious anomalies "
                "or inconsistent values and return a JSON-like dict with keys: Valid (bool), Comment (str). "
                f"Record: {rec}"
            )

            # safe call -> either real response or fallback
            resp = safe_llm_predict(self._llm, prompt, fallback_type="audit")

            # Normalize response: if LLM returned dict-like or string, try to coerce
            if isinstance(resp, dict):
                audit = resp
            else:
                # If it's a string, try to parse for simple patterns; otherwise default
                try:
                    # attempt eval if it looks like a dict
                    audit = eval(resp) if resp.strip().startswith("{") else {"Valid": True, "Comment": str(resp)}
                except Exception:
                    audit = {"Valid": True, "Comment": str(resp)}

            # Attach audit results to the record copy (avoid mutating original)
            rec_copy = dict(rec)
            rec_copy["FD_Valid"] = bool(audit.get("Valid", True))
            rec_copy["FD_Comment"] = str(audit.get("Comment", "No comment"))
            reviewed.append(rec_copy)

        logging.info(f"FDAuditorAgent: reviewed {len(reviewed)} records.")
        return reviewed


class CentralizedRiskAgent(Agent):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def __init__(self, llm, name="CentralizedRiskAgent"):
        super().__init__(name=name)
        self._llm = llm

    # expects inputs: reviewed_data (list of dict)
    def run(self, reviewed_data):
        final = []
        for rec in reviewed_data:
            prompt = (
                "You are a clinical risk assessor. Based on the patient record and auditor feedback, "
                "respond with a JSON-like dict containing FinalRisk: 'High' or 'Low'. Only output a dict.\n"
                f"Record: {rec}"
            )

            resp = safe_llm_predict(self._llm, prompt, fallback_type="risk")

            # Normalize
            if isinstance(resp, dict):
                risk_dict = resp
            else:
                try:
                    risk_dict = eval(resp) if resp.strip().startswith("{") else {"FinalRisk": str(resp).strip()}
                except Exception:
                    # fallback random
                    risk_dict = {"FinalRisk": np.random.choice(["High", "Low"])}

            rec_copy = dict(rec)
            rec_copy["FinalRisk"] = str(risk_dict.get("FinalRisk", np.random.choice(["High", "Low"])))
            final.append(rec_copy)

        logging.info(f"CentralizedRiskAgent: produced {len(final)} final records.")
        return final


In [ ]:
# Cell 5: Initialize Agents, App, Runner
patient_agent = PatientInputAgent(gemini_llm)
fd_agent = FDAuditorAgent(gemini_llm)
central_agent = CentralizedRiskAgent(gemini_llm)

# Create ADK App with initial agent
#from google.ai.generativelanguage import App
app = App(
    name="DiabetesRiskApp",
    root_agent=patient_agent   # ⭐ REQUIRED
)
runner = InMemoryRunner(app=app)
print("✅ App and InMemoryRunner initialized.")


In [ ]:
# Cell 6: Run the sequential workflow via runner (App.agent is changed between steps)
# Step 1: generate patient inputs
risk_agent = CentralizedRiskAgent(gemini_llm)
patients = patient_agent.run(num_patients=20)
print(f"Generated patients: {len(patients)}")

#patients = patient_agent.run()
audited  = fd_agent.run(patients)
risk     = risk_agent.run(audited)



In [ ]:
class DummyRiskModel:
    def predict(self, patient):
        # Simple formula combining glucose & BMI (you can improve this later)
        return (patient["Blood Glucose"] / 200) + (patient["BMI"] / 100)
        
model = DummyRiskModel()

In [ ]:
final_records = []

for patient in patients:
    risk = model.predict(patient)
    final_records.append({
        "patient_id": patient["PatientID"],
        "risk_score": risk,
        "glucose": patient["Blood Glucose"],
        "bmi": patient["BMI"],
        "bp": patient["BP"],
        "cholesterol": patient["Cholesterol"],
        "meals": patient["Meals"],
    })



In [57]:
final_df = pd.DataFrame(final_records)

csv_path = "/kaggle/working/centralized_risk_predictions.csv"
final_df.to_csv(csv_path, index=False)

print(f"✅ CSV saved to: {csv_path}")

final_df.head()

✅ CSV saved to: /kaggle/working/centralized_risk_predictions.csv


,patient_id,risk_score,glucose,bmi,bp,cholesterol,meals
0,1,1.045,160,24.5,123/72,292,High Protein
1,2,0.636,89,19.1,104/70,269,High Protein
2,3,0.545,70,19.5,156/87,167,Vegan
3,4,0.944,146,21.4,131/69,213,Balanced
4,5,1.276,186,34.6,106/76,165,Balanced
